# Ex. 3.2 HBASE
We use the *HappyBase* lib: https://happybase.readthedocs.io/en/latest/installation.html

In [ ]:
! conda install -c conda-forge -y happybase
#! pip install -c conda-forge  happybase

/bin/bash: conda: command not found


In [1]:
!pip install happybase

     |████████████████████████████████| 40kB 2.0MB/s 
     |████████████████████████████████| 368kB 18.2MB/s 
     |████████████████████████████████| 51kB 5.8MB/s 
  Created wheel for happybase: filename=happybase-1.2.0-py2.py3-none-any.whl size=26622 sha256=9a3133a4dcb22d857d4c08afb31b9ddb217857e405dec048c4e90db6e5747990
  Stored in directory: /root/.cache/pip/wheels/91/43/5c/8bf59c8c579f2b344f30fd013bdfd5b526adaf694c79a355ae
  Created wheel for thriftpy2: filename=thriftpy2-0.4.14-cp37-cp37m-linux_x86_64.whl size=940295 sha256=86b2570213a16514ba9454a82c5150aa40fd2d3691c299b9742a4fad7e80d41e
  Stored in directory: /root/.cache/pip/wheels/89/c6/6b/c94a9a90153934a39a26ed4d94254d0e347b706f989b526c8b
Successfully built happybase thriftpy2


In [2]:
# imports
import happybase

In [3]:
# try to connect
server="ec2-34-254-248-152.eu-west-1.compute.amazonaws.com" #need to change this for every lab-session
connection = happybase.Connection(server)

In [4]:
families = {
    'cf1': dict(max_versions=10),
    'cf2': dict(max_versions=1, block_cache_enabled=False),
    'cf3': dict(),  # use defaults
}

In [ ]:
#connection.create_table('lab3', families)  # table allredy exists

In [5]:
print(connection.tables())

[b'49a5b1a_employees', b'dh_user_id_employees', b'employee_Group2', b'employee_Group_2', b'employee_group_five', b'employees_smunaret', b'group1_employee', b'group3_user_id_employees', b'lab3', b'mspitzna_employees', b'tm_user_id_employees', b'user_id_employees']


In [24]:
#connect to a table
table = connection.table('lab3')

In [7]:
#see column families
table.families()

{b'cf1': {'block_cache_enabled': False,
  'bloom_filter_nb_hashes': 0,
  'bloom_filter_type': b'NONE',
  'bloom_filter_vector_size': 0,
  'compression': b'NONE',
  'in_memory': False,
  'max_versions': 10,
  'name': b'cf1:',
  'time_to_live': 2147483647},
 b'cf2': {'block_cache_enabled': False,
  'bloom_filter_nb_hashes': 0,
  'bloom_filter_type': b'NONE',
  'bloom_filter_vector_size': 0,
  'compression': b'NONE',
  'in_memory': False,
  'max_versions': 1,
  'name': b'cf2:',
  'time_to_live': 2147483647},
 b'cf3': {'block_cache_enabled': False,
  'bloom_filter_nb_hashes': 0,
  'bloom_filter_type': b'NONE',
  'bloom_filter_vector_size': 0,
  'compression': b'NONE',
  'in_memory': False,
  'max_versions': 3,
  'name': b'cf3:',
  'time_to_live': 2147483647}}

In [8]:
#insert something
table.put('row 6', {'cf1:col7': '17.33', 'cf1:col8': '344'})

In [9]:
table.families()

{b'cf1': {'block_cache_enabled': False,
  'bloom_filter_nb_hashes': 0,
  'bloom_filter_type': b'NONE',
  'bloom_filter_vector_size': 0,
  'compression': b'NONE',
  'in_memory': False,
  'max_versions': 10,
  'name': b'cf1:',
  'time_to_live': 2147483647},
 b'cf2': {'block_cache_enabled': False,
  'bloom_filter_nb_hashes': 0,
  'bloom_filter_type': b'NONE',
  'bloom_filter_vector_size': 0,
  'compression': b'NONE',
  'in_memory': False,
  'max_versions': 1,
  'name': b'cf2:',
  'time_to_live': 2147483647},
 b'cf3': {'block_cache_enabled': False,
  'bloom_filter_nb_hashes': 0,
  'bloom_filter_type': b'NONE',
  'bloom_filter_vector_size': 0,
  'compression': b'NONE',
  'in_memory': False,
  'max_versions': 3,
  'name': b'cf3:',
  'time_to_live': 2147483647}}

In [26]:
#read table
res=table.scan()
for key,row in res:
    print(key,row)

BrokenPipeError: ignored

# Ex 1
Employee data in HBASE.
* create a new table to hold the employee data -> use a distinct name, like ``user_id_employees``
* How would you create the column families?
* read data from ``employees.csv`` and put it into the table ->  use  ``import csv`` to read the data -> [API docs](https://docs.python.org/3/library/csv.html)
* use batch-processing to enter more than one entry per transaction
* alter some entries of the table with ``timestamps``
* perform some sample scans for the latest entries

In [15]:
connection.create_table('1234_user_id_employees', families)
print(connection.tables())
table = connection.table('1234_user_id_employees')

[b'1234_user_id_employees', b'49a5b1a_employees', b'dh_user_id_employees', b'employee_Group2', b'employee_Group_2', b'employee_group_five', b'employees_smunaret', b'group1_employee', b'group3_user_id_employees', b'lab3', b'mspitzna_employees', b'tm_user_id_employees', b'user_id_employees']


In [16]:
table = connection.table('1234_user_id_employees')

In [17]:
table.families()

{b'cf1': {'block_cache_enabled': False,
  'bloom_filter_nb_hashes': 0,
  'bloom_filter_type': b'NONE',
  'bloom_filter_vector_size': 0,
  'compression': b'NONE',
  'in_memory': False,
  'max_versions': 10,
  'name': b'cf1:',
  'time_to_live': 2147483647},
 b'cf2': {'block_cache_enabled': False,
  'bloom_filter_nb_hashes': 0,
  'bloom_filter_type': b'NONE',
  'bloom_filter_vector_size': 0,
  'compression': b'NONE',
  'in_memory': False,
  'max_versions': 1,
  'name': b'cf2:',
  'time_to_live': 2147483647},
 b'cf3': {'block_cache_enabled': False,
  'bloom_filter_nb_hashes': 0,
  'bloom_filter_type': b'NONE',
  'bloom_filter_vector_size': 0,
  'compression': b'NONE',
  'in_memory': False,
  'max_versions': 3,
  'name': b'cf3:',
  'time_to_live': 2147483647}}

In [21]:
import csv
with open('employees.csv', newline='') as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    i = 0;
    batch = table.batch(timestamp=None)
    for row in reader:
      batch.put(row[0], {'cf1:col1': row[1], 'cf1:col2': row[2], 'cf1:col3': row[3], 'cf1:col4': row[4], 'cf2:col5': row[5]})
      i=i+1
      if i>150:
        break;
batch.send()

In [22]:
#read table
#res=table.scan(include_timestamp=True, timestamp=1234567890)
res=table.scan()
for key,row in res:
    print(key,row)

b'10001' {b'cf1:col1': b"'1953-09-02'", b'cf1:col2': b"'Georgi'", b'cf1:col3': b"'Facello'", b'cf1:col4': b"'M'", b'cf2:col5': b"'1986-06-26'"}
b'10002' {b'cf1:col1': b"'1964-06-02'", b'cf1:col2': b"'Bezalel'", b'cf1:col3': b"'Simmel'", b'cf1:col4': b"'F'", b'cf2:col5': b"'1985-11-21'"}
b'10003' {b'cf1:col1': b"'1959-12-03'", b'cf1:col2': b"'Parto'", b'cf1:col3': b"'Bamford'", b'cf1:col4': b"'M'", b'cf2:col5': b"'1986-08-28'"}
b'10004' {b'cf1:col1': b"'1954-05-01'", b'cf1:col2': b"'Chirstian'", b'cf1:col3': b"'Koblick'", b'cf1:col4': b"'M'", b'cf2:col5': b"'1986-12-01'"}
b'10005' {b'cf1:col1': b"'1955-01-21'", b'cf1:col2': b"'Kyoichi'", b'cf1:col3': b"'Maliniak'", b'cf1:col4': b"'M'", b'cf2:col5': b"'1989-09-12'"}
b'10006' {b'cf1:col1': b"'1953-04-20'", b'cf1:col2': b"'Anneke'", b'cf1:col3': b"'Preusig'", b'cf1:col4': b"'F'", b'cf2:col5': b"'1989-06-02'"}
b'10007' {b'cf1:col1': b"'1957-05-23'", b'cf1:col2': b"'Tzvetan'", b'cf1:col3': b"'Zielinski'", b'cf1:col4': b"'F'", b'cf2:col5': b"